In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from statistics import mode
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv('sentiment_data/grand_df.csv', index_col=0)
df.head()

,album,anger,anticipation,api_path,disgust,fear,joy,lyrics,negative,polarity,positive,primary_artist,release_date,sadness,song_endpoint,surprise,title,trust,url
0,Boy In Detention (Mixtape),14.0,13.0,/songs/62387,5.0,7.0,12.0,"Beats I be servin', Saks Fifth splurgin' Diam...",11.0,-0.9809,24.0,Chris Brown,2011-08-05,5.0,http://api.genius.com/songs/62387,11.0,100%,17.0,https://genius.com/Chris-brown-100-lyrics
1,Boy In Detention (Mixtape),19.0,1.0,/songs/54007,16.0,15.0,1.0,We in the motherfuckin' buildin' 100 fuckin...,20.0,-0.9967,13.0,Chris Brown,2011-08-05,16.0,http://api.genius.com/songs/54007,1.0,100 Bottles,4.0,https://genius.com/Chris-brown-100-bottles-lyrics
2,X,2.0,4.0,/songs/516349,2.0,1.0,6.0,You got me doing 101 on the 101 Don't go nowh...,6.0,0.9516,11.0,Chris Brown,2014-09-16,2.0,http://api.genius.com/songs/516349,2.0,101 (Interlude),6.0,https://genius.com/Chris-brown-101-interlude-l...
3,None,0.0,0.0,/songs/3103904,0.0,4.0,0.0,Ain't no way we could ignore this fire When I...,3.0,0.8592,4.0,Chris Brown,2017-05-31,2.0,http://api.genius.com/songs/3103904,0.0,10 Feet,2.0,https://genius.com/Chris-brown-10-feet-lyrics
4,None,20.0,17.0,/songs/57217,11.0,12.0,14.0,"Welcome to the Cypher Myself, K-Mac, Ace Hood...",23.0,0.9745,39.0,Chris Brown,2019-04-04,9.0,http://api.genius.com/songs/57217,17.0,2011 BET Cypher Chris Brown and Friends,25.0,https://genius.com/Chris-brown-2011-bet-cypher...


In [3]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

In [4]:
df = df.drop(df[df['release_year'] < 1900].index, axis=0)

In [5]:
df = df.drop(df[df['release_year'] > 2019].index, axis=0)

In [6]:
df

,album,anger,anticipation,api_path,disgust,fear,joy,lyrics,negative,polarity,positive,primary_artist,release_date,sadness,song_endpoint,surprise,title,trust,url,release_year
0,Boy In Detention (Mixtape),14.0,13.0,/songs/62387,5.0,7.0,12.0,"Beats I be servin', Saks Fifth splurgin' Diam...",11.0,-0.9809,24.0,Chris Brown,2011-08-05,5.0,http://api.genius.com/songs/62387,11.0,100%,17.0,https://genius.com/Chris-brown-100-lyrics,2011.0
1,Boy In Detention (Mixtape),19.0,1.0,/songs/54007,16.0,15.0,1.0,We in the motherfuckin' buildin' 100 fuckin...,20.0,-0.9967,13.0,Chris Brown,2011-08-05,16.0,http://api.genius.com/songs/54007,1.0,100 Bottles,4.0,https://genius.com/Chris-brown-100-bottles-lyrics,2011.0
2,X,2.0,4.0,/songs/516349,2.0,1.0,6.0,You got me doing 101 on the 101 Don't go nowh...,6.0,0.9516,11.0,Chris Brown,2014-09-16,2.0,http://api.genius.com/songs/516349,2.0,101 (Interlude),6.0,https://genius.com/Chris-brown-101-interlude-l...,2014.0
3,None,0.0,0.0,/songs/3103904,0.0,4.0,0.0,Ain't no way we could ignore this fire When I...,3.0,0.8592,4.0,Chris Brown,2017-05-31,2.0,http://api.genius.com/songs/3103904,0.0,10 Feet,2.0,https://genius.com/Chris-brown-10-feet-lyrics,2017.0
4,None,20.0,17.0,/songs/57217,11.0,12.0,14.0,"Welcome to the Cypher Myself, K-Mac, Ace Hood...",23.0,0.9745,39.0,Chris Brown,2019-04-04,9.0,http://api.genius.com/songs/57217,17.0,2011 BET Cypher Chris Brown and Friends,25.0,https://genius.com/Chris-brown-2011-bet-cypher...,2019.0
5,Fortune,3.0,11.0,/songs/79425,1.0,12.0,11.0,Faire l'amour lendemain What if the sky was...,8.0,0.8478,14.0,Chris Brown,2012-06-29,2.0,http://api.genius.com/songs/79425,2.0,2012,4.0,https://genius.com/Chris-brown-2012-lyrics,2012.0
6,None,8.0,10.0,/songs/60609,8.0,11.0,14.0,Shattered glasses and cigarettes There's a lo...,37.0,-0.9576,13.0,Chris Brown,2012-06-19,9.0,http://api.genius.com/songs/60609,13.0,2 Complicated,13.0,https://genius.com/Chris-brown-2-complicated-l...,2012.0
7,None,6.0,3.0,/songs/3089879,5.0,2.0,15.0,"Oh, oh, oh, babe, yeah I know what you need, ...",13.0,-0.1984,47.0,Chris Brown,2017-05-28,3.0,http://api.genius.com/songs/3089879,3.0,2 the Side,23.0,https://genius.com/Chris-brown-2-the-side-lyrics,2017.0
8,None,16.0,7.0,/songs/2404874,16.0,15.0,4.0,And I'm like Jordan wearin' 45 I told you I w...,27.0,-0.9491,18.0,Chris Brown,2017-05-31,20.0,http://api.genius.com/songs/2404874,7.0,45,26.0,https://genius.com/Chris-brown-45-lyrics,2017.0
9,Fan of a Fan,13.0,10.0,/songs/2877,9.0,9.0,7.0,"Yeah, leggo It's what they wanted right? Bl...",15.0,-0.7584,15.0,Chris Brown,2010-05-16,9.0,http://api.genius.com/songs/2877,7.0,48 Bar Rap,9.0,https://genius.com/Chris-brown-48-bar-rap-lyrics,2010.0


In [7]:
df.shape

(43781, 20)

In [8]:
count = df.groupby('primary_artist')[['title']].count().sort_values(by='title', ascending=False)

In [9]:
count.title.mean()

130.6865671641791

In [10]:
artists_with_too_many_songs = count[count.title > 200].index.tolist()

In [11]:
for artist in artists_with_too_many_songs:
    filtered_df = df[df['primary_artist'] == artist]
    filtered_df = filtered_df.sample(200)
    df.drop(df[df['primary_artist'] == artist].index, axis=0, inplace=True)
    df = df.append(filtered_df)

In [12]:
df.drop(df[df['release_date'].isna()].index, axis=0, inplace=True)

In [13]:
df.shape

(35915, 20)

In [14]:
artists_count = df.groupby('release_year')[['primary_artist']].nunique().reset_index()

In [15]:
df_normalized = df.groupby('release_year')[['anger', 'positive', 'negative', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']].sum()

In [16]:
df_normalized = df_normalized.reset_index()

In [17]:
df_normalized.head()

,release_year,anger,positive,negative,anticipation,disgust,fear,joy,sadness,surprise,trust
0,1961.0,2.0,15.0,6.0,6.0,2.0,0.0,13.0,2.0,2.0,14.0
1,1962.0,20.0,85.0,50.0,26.0,24.0,32.0,60.0,42.0,18.0,42.0
2,1963.0,76.0,266.0,181.0,196.0,74.0,65.0,200.0,76.0,60.0,116.0
3,1964.0,109.0,529.0,243.0,209.0,100.0,113.0,384.0,158.0,107.0,255.0
4,1965.0,127.0,570.0,311.0,210.0,117.0,175.0,340.0,218.0,133.0,346.0


In [18]:
sums = df_normalized['anger'] + df_normalized['positive'] + df_normalized['negative'] + df_normalized['anticipation'] + df_normalized['disgust'] + df_normalized['fear'] + df_normalized['joy'] + df_normalized['sadness'] + df_normalized['surprise'] + df_normalized['trust']
df_normalized['sum'] = sums

In [19]:
cols = list(df_normalized.columns)
cols.remove('release_year')
for col in df_normalized.columns:
    df_normalized[col] = df_normalized[col].apply(int)

In [20]:
cols.remove('sum')
for col in cols:
    df_normalized[col] = df_normalized[col] / df_normalized['sum']

In [21]:
artists_count

,release_year,primary_artist
0,1961.0,1
1,1962.0,2
2,1963.0,3
3,1964.0,3
4,1965.0,4
5,1966.0,4
6,1967.0,4
7,1968.0,4
8,1969.0,7
9,1970.0,10


In [22]:
df_normalized

,release_year,anger,positive,negative,anticipation,disgust,fear,joy,sadness,surprise,trust,sum
0,1961,0.032258,0.241935,0.096774,0.096774,0.032258,0.000000,0.209677,0.032258,0.032258,0.225806,62
1,1962,0.050125,0.213033,0.125313,0.065163,0.060150,0.080201,0.150376,0.105263,0.045113,0.105263,399
2,1963,0.058015,0.203053,0.138168,0.149618,0.056489,0.049618,0.152672,0.058015,0.045802,0.088550,1310
3,1964,0.049388,0.239692,0.110104,0.094699,0.045310,0.051201,0.173992,0.071590,0.048482,0.115541,2207
4,1965,0.049863,0.223793,0.122104,0.082450,0.045936,0.068708,0.133490,0.085591,0.052218,0.135846,2547
5,1966,0.035322,0.219780,0.111460,0.144427,0.029827,0.043171,0.149922,0.057300,0.080848,0.127943,1274
6,1967,0.050294,0.220631,0.130323,0.092511,0.055433,0.062775,0.132159,0.070852,0.057269,0.127753,2724
7,1968,0.060809,0.198981,0.136262,0.107927,0.046482,0.073225,0.120025,0.081503,0.057307,0.117479,3141
8,1969,0.054643,0.213214,0.105000,0.114643,0.037857,0.070357,0.137143,0.065714,0.062500,0.138929,2800
9,1970,0.041909,0.240288,0.108902,0.108749,0.042062,0.056745,0.151117,0.075558,0.049709,0.124962,6538


In [23]:
df_normalized = df_normalized.drop(df_normalized[df_normalized['release_year'] < 2000].index, axis=0)
artists_count = artists_count.drop(artists_count[artists_count['release_year'] < 2000].index, axis=0)

In [24]:
df_original = df.copy()
df = df_normalized

In [25]:
df_original.head()

,album,anger,anticipation,api_path,disgust,fear,joy,lyrics,negative,polarity,positive,primary_artist,release_date,sadness,song_endpoint,surprise,title,trust,url,release_year
588,Feed the Machine,6.0,18.0,/songs/3101307,4.0,5.0,10.0,"All your life, scramble and scurry Take your ...",24.0,0.9869,23.0,Nickelback,2017-06-16,18.0,http://api.genius.com/songs/3101307,1.0,After the Rain,16.0,https://genius.com/Nickelback-after-the-rain-l...,2017.0
589,All the Right Reasons,4.0,8.0,/songs/196492,3.0,8.0,7.0,"I, I'm driving black on black Just got my lic...",17.0,-0.9644,11.0,Nickelback,2005-11-08,9.0,http://api.genius.com/songs/196492,7.0,Animals,7.0,https://genius.com/Nickelback-animals-lyrics,2005.0
590,The Long Road,10.0,1.0,/songs/196477,8.0,10.0,2.0,"I'm through, thinking of things to say to you...",23.0,-0.9964,6.0,Nickelback,2004-03-21,18.0,http://api.genius.com/songs/196477,18.0,Another Hole in the Head,2.0,https://genius.com/Nickelback-another-hole-in-...,2004.0
591,The Long Road,1.0,8.0,/songs/196470,3.0,11.0,1.0,"Hands on the mirror, can't get much clearer C...",9.0,0.9634,3.0,Nickelback,2004-03-21,6.0,http://api.genius.com/songs/196470,1.0,Because of You,2.0,https://genius.com/Nickelback-because-of-you-l...,2004.0
592,The Long Road,4.0,6.0,/songs/196467,1.0,4.0,10.0,"Believe it or not, everyone has things that t...",4.0,0.9931,23.0,Nickelback,2004-03-21,3.0,http://api.genius.com/songs/196467,0.0,Believe It or Not,20.0,https://genius.com/Nickelback-believe-it-or-no...,2004.0


In [26]:
polarity = df_original.groupby('release_year')[['polarity']].mean().reset_index()
polarity = polarity.drop(polarity[polarity['release_year'] < 2000].index, axis=0)

In [39]:
polarity['text'] = round(polarity['polarity'],2)

In [50]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import HoverTool, Legend, ColumnDataSource, Panel, Tabs, ColumnDataSource, LabelSet

a = int(df['release_year'].min())
b = int(df['release_year'].max()) 
p = figure(plot_width=1000, plot_height=300, x_range=(2000, 2020), y_range=(0.1, 0.5), x_axis_label='Years',
           toolbar_location='above')

r0 = p.line(polarity['release_year'],polarity['polarity'], color='black', line_width=2, legend="composite polarity score")
r1 = p.circle(polarity['release_year'], polarity['polarity'], fill_color='white', size=10, color='black', legend="composite polarity score")


p.circle(polarity['release_year'], polarity['polarity'], size=20, fill_color='gray', hover_fill_color='gray',
               fill_alpha=0.02, hover_alpha=0.2, line_color=None, hover_line_color='gray')

source = ColumnDataSource(polarity)

labels = LabelSet(x='release_year', y='polarity', text='text', source=source, y_offset=13, text_font_size="12pt", text_color="#555555", text_align='center')
p.add_layout(labels)

TOOLTIPS = """
<div>
    <div>
        <span style="font-size: 15px;">Year: </span>
        <span style="font-size: 17px; font-weight: bold;">$x{0}</span>
    </div>
</div>
"""
p.add_tools(HoverTool(tooltips=TOOLTIPS, mode='mouse'))
p.background_fill_color = "#f2f2f2"
p.grid.grid_line_color = "white"
p.xaxis.major_tick_line_color = 'firebrick'
p.xaxis.major_tick_line_width = 5
p.xaxis.minor_tick_line_color = 'orange'
p.axis.major_tick_out = 10
p.axis.minor_tick_out = 8
p.xgrid[0].ticker.desired_num_ticks = 10

show(p)